In [ ]:
from abismal.command_line.parser.__init__ import groups
import ipywidgets as widgets

class Text(widgets.Box):
    def __init__(self, **kwargs):
        description = ''
        if 'description' in kwargs:
            description = kwargs.pop('description')
        children = [
            widgets.Label(description),
            widgets.Text(**kwargs),
        ]
        super().__init__(children)

def arg_to_name(*args, **kwargs):
    if 'metavar' in kwargs:
        return metavar
    name = args[-1]
    name = name.lstrip('-').replace('-', '_')
    return name

def arg_to_widget(*args, **kwargs):
    """
    Convert an Abismal argument into a widget
    """
    name = arg_to_name(*args, **kwargs)
    if 'action' in kwargs:
        if kwargs['action'] == 'store_true':
            return widgets.ToggleButton(
                value=False,
                description=name,
                disabled=False,
                button_style='', # 'success', 'info', 'warning', 'danger' or ''
                tooltip=kwargs.get('help', ''),
            )
    # Fallback text field
    default = kwargs.get('default', '')
    widget = Text(
        placeholder=str(default),
        tooltip=kwargs.get('help', ''),
        description=name,
    )
    return widget

def is_required(*args, **kwargs):
    if args[0][0] != '-':
        return True
    elif 'required' in kwargs and kwargs['required']:
        return True
    return False

children = []
cl_args = {}
required_widgets = []

for group_name,group in groups.items():
    group_args = []
    group_widgets = []
    for args,kwargs in group.args_and_kwargs:
        name = arg_to_name(*args, **kwargs)
        widget = arg_to_widget(*args, **kwargs)
        if is_required(*args, **kwargs):
            required_widgets.append(widget)
        else:
            group_widgets.append(widget)
        cl_args[name] = widget
    group_box = widgets.VBox(group_widgets)
    children.append(group_box)
required_box = widgets.VBox(required_widgets)

tab = widgets.Tab(
    children = [required_box] + children,
    titles = ['Required'] + list(groups.keys())
)
tab